### 0. Library import 

In [169]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from webdriver_manager.chrome import ChromeDriverManager
print(selenium.__version__)

4.10.0


### 1. URL setting and Chromedriver test

In [170]:
import requests

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
res = requests.get(url)

res.status_code # HTTP 상태 코드

403

In [171]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

### 2. 필수 과제 
1페이지의 60개의 코디맵 정보 가져오기

In [173]:
# chromedriver settings

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

driver.get(url) 
driver.implicitly_wait(2)
try:
    driver.maximize_window() 
except:
    pass

interval = 1    # 1초에 한 번씩 스크롤 내림
prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height
    
    
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']       # 코디맵 카테고리, 코디맵 제목, 등록 날짜, 조회수, 댓글수 크롤링
values = []                                            
soup = BeautifulSoup(driver.page_source, 'lxml')        
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})


for i, row in enumerate(data_rows): 

    print('{}번째 코디맵 크롤링'.format(i+1))   
    blank = []

    category = row.find('span', attrs={'class':'style-list-information__text'})       # 코디맵 카테고리 가져오기
    if category:                                                                    
        category = category.get_text().strip()                                
        blank.append(category)
    else:                                                                   
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올 때 문제 발생'.format(i+1))
        continue


    title = row.find('strong', attrs={'class':'style-list-information__title'})      # 코디맵 제목 정보 가져오기
    if title:
        title = title.get_text().strip()
        blank.append(title)
    else:
        blank.append('Something is wrong')
        print('{}번째 제목 가져올 때 문제 발생'.format(i+1))
        continue


    info = row.find_all('span', attrs={'class':'post-information__text'})        # 등록 날짜, 조회수, 댓글수 포함한 sub html 크롤링
   
    if info:
        date = info[0].get_text().strip()       # 인덱싱을 이용해 코디맵 등록 날짜 가져오기
        blank.append(date)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜 가져올 때 문제 발생'.format(i+1))
        continue
    
    
    if info:
        view = info[1].get_text().strip().split(' ')[-1]        # 인덱싱을 이용해 코디맵 조회수 가져오기
        blank.append(view)
    else:
        blank.append('Something is wrong')
        print('{}번째 조회수 가져올 때 문제 발생'.format(i+1))
        continue
    
    
    if info:
        reply = info[2].get_text().strip().split(' ')[-1]       # 인덱싱을 이용해 코디맵 댓글수 가져오기
        blank.append(reply)
    else:
        blank.append('Something is wrong')
        print('{}번째 댓글수 가져올 때 문제 발생'.format(i+1))
        continue
    
    values.append(blank)
    print('---------------------------------------------------')
    
df = pd.DataFrame(values, columns = columns)

1번째 코디맵 크롤링
---------------------------------------------------
2번째 코디맵 크롤링
---------------------------------------------------
3번째 코디맵 크롤링
---------------------------------------------------
4번째 코디맵 크롤링
---------------------------------------------------
5번째 코디맵 크롤링
---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------

In [174]:
# dataframe 확인

df.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,092",49
1,캐주얼,블랙 매니아,21.02.23,"34,681",46
2,캐주얼,오늘의 코디,22.03.22,"12,205",38
3,댄디,추천 댄디 룩,21.09.15,"20,191",27
4,캐주얼,릴랙스 룩,21.02.23,"23,544",22
5,스트릿,힙하다 힙해,21.02.03,"35,295",21
6,캐주얼,심플한 코디,21.07.30,"39,591",18
7,댄디,데이트 코디,21.12.24,"10,316",18
8,댄디,훈훈 남친 룩,21.03.24,"19,350",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,219",16


In [176]:
# dataframe 크기 확인 (5개 columns, 60개의 코디맵)

df.shape

(60, 5)

In [177]:
# save dataframe as csv

df.to_csv('정지원_크롤링_필수과제.csv', encoding = 'utf-8-sig')

### 3. Challenge 과제

#### 3.1. 1페이지부터 5페이지까지의 코디맵 정보 가져오기 (5 columms)

In [146]:
# click 대신 send_keys 이용을 위한 library import

from selenium.webdriver.common.keys import Keys

In [190]:
# chromedriver settings

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # 코드가 돌아갈 때마다 크롬 버전에 맞는 드라이버 재설치 해줌

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

driver.get(url) 
driver.implicitly_wait(2) 
try:
    driver.maximize_window() 
except:
    pass

codimap_list = []

# 1~5페이지 크롤링을 위한 for문 삽입

for page in range(5):
    
    print('{}페이지 크롤링 시작!'.format(page+1))
    interval = 1
    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

            # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height
        
        
    columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']       
    values = []                                            
    soup = BeautifulSoup(driver.page_source, 'lxml')        
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'})


    for i, row in enumerate(data_rows):                    

        print('{0}페이지 {1}번째 코디맵 크롤링'.format(page+1, i+1))
        blank = []

        category = row.find('span', attrs={'class':'style-list-information__text'})      
        if category:                                                               
            category = category.get_text().strip()                                          
            blank.append(category)
        else:                                                                       
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 카테고리 가져올 때 문제 발생'.format(page+1, i+1))
            continue


        title = row.find('strong', attrs={'class':'style-list-information__title'})      
        if title:
            title = title.get_text().strip()
            blank.append(title)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 제목 가져올 때 문제 발생'.format(page+1, i+1))
            continue


        info = row.find_all('span', attrs={'class':'post-information__text'})        
    
        if info:
            date = info[0].get_text().strip()    
            blank.append(date)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 날짜 가져올 때 문제 발생'.format(page+1, i+1))
            continue
        
        
        if info:
            view = info[1].get_text().strip().split(' ')[-1]    
            blank.append(view)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 조회수 가져올 때 문제 발생'.format(page+1, i+1))
            continue
        
        
        if info:
            reply = info[2].get_text().strip().split(' ')[-1]    
            blank.append(reply)
        else:
            blank.append('Something is wrong')
            print('{0}페이지 {1}번째 댓글수 가져올 때 문제 발생'.format(page+1, i+1))
            continue

        values.append(blank)        
        print('---------------------------------------------------')
    
    path = '/html/body/div[3]/div[3]/form/div[4]/div/div[4]/div/div/a[{}]'.format(str(page+4))      # 다음 페이지로 넘어가는 Xpath 
    next_page_button = driver.find_element(By.XPATH, path)      # 다음 페이지 클릭
    next_page_button.send_keys(Keys.ENTER)      # click 함수가 오류가 나서 send_keys로 변경
    time.sleep(3)    
        
    df = pd.DataFrame(values, columns = columns)
    codimap_list.append(df)

1페이지 크롤링 시작!
1페이지 1번째 코디맵 크롤링
---------------------------------------------------
1페이지 2번째 코디맵 크롤링
---------------------------------------------------
1페이지 3번째 코디맵 크롤링
---------------------------------------------------
1페이지 4번째 코디맵 크롤링
---------------------------------------------------
1페이지 5번째 코디맵 크롤링
---------------------------------------------------
1페이지 6번째 코디맵 크롤링
---------------------------------------------------
1페이지 7번째 코디맵 크롤링
---------------------------------------------------
1페이지 8번째 코디맵 크롤링
---------------------------------------------------
1페이지 9번째 코디맵 크롤링
---------------------------------------------------
1페이지 10번째 코디맵 크롤링
---------------------------------------------------
1페이지 11번째 코디맵 크롤링
---------------------------------------------------
1페이지 12번째 코디맵 크롤링
---------------------------------------------------
1페이지 13번째 코디맵 크롤링
---------------------------------------------------
1페이지 14번째 코디맵 크롤링
---------------------------------------------------
1페이지 15번째 코디맵 크롤

In [191]:
# 페이지별로 dataframe concat & 확인

df_try = pd.concat(codimap_list).reset_index(drop=True) # index 초기화하고 codimap_list의 페이지별 df 연결
df_try.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,092",49
1,캐주얼,블랙 매니아,21.02.23,"34,681",46
2,캐주얼,오늘의 코디,22.03.22,"12,205",38
3,댄디,추천 댄디 룩,21.09.15,"20,191",27
4,캐주얼,릴랙스 룩,21.02.23,"23,544",22
5,스트릿,힙하다 힙해,21.02.03,"35,295",21
6,캐주얼,심플한 코디,21.07.30,"39,591",18
7,댄디,데이트 코디,21.12.24,"10,316",18
8,댄디,훈훈 남친 룩,21.03.24,"19,350",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,219",16


In [194]:
# dataframe 크기 확인 (5개 columns, 300개(60*5)의 코디맵)

df_try.shape

(300, 5)

#### 3.2. Challenge 과제 : 1페이지부터 5페이지까지의 코디맵 설명, 해시태그, 이미지 url을 추가하여 코디맵 정보 가져오기 (8 columms)

In [195]:
# chromedriver settings

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) 

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

driver.get(url) 
driver.implicitly_wait(2) 
try:
    driver.maximize_window() 
except:
    pass

codimap_list = []


# 1~5페이지 크롤링을 위한 for문 삽입

for page in range(5):
    
    print('{}페이지 크롤링 시작!/n'.format(page+1))
    
    interval = 1
    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤을 가장 아래로 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    # 코디맵 카테고리, 코디맵 제목, 등록 날짜, 조회수, 댓글수, 코디맵 설명, 해시태그, 이미지 url 크롤링
    columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']       
    values = []                                            
    soup = BeautifulSoup(driver.page_source, 'lxml')        
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'})


    for i, row in enumerate(data_rows):                     

        print('{0}페이지 {1}번째 코디맵 크롤링'.format(page+1, i+1))
        blank = []

        category = row.find('span', attrs={'class':'style-list-information__text'})         # 코디맵 카테고리 가저오기
        if category:                                                                    
            category = category.get_text().strip()                                          
            blank.append(category)
        else:                                                                      
            blank.append('Something is wrong')
            print('{}번째 카테고리 가져올 때 문제 발생'.format(i+1))
            continue


        title = row.find('strong', attrs={'class':'style-list-information__title'})         # 코디맵 제목 가져오기
        if title:
            title = title.get_text().strip()
            blank.append(title)
        else:
            blank.append('Something is wrong')
            print('{}번째 제목 가져올 때 문제 발생'.format(i+1))
            continue


        info = row.find_all('span', attrs={'class':'post-information__text'})         # 등록 날짜, 조회수, 댓글수 포함한 sub html 크롤링
    
        if info:
            date = info[0].get_text().strip()       # 인덱싱을 사용해 코디맵 등록 날짜 가져오기
            blank.append(date)
        else:
            blank.append('Something is wrong')
            print('{}번째 날짜 가져올 때 문제 발생'.format(i+1))
            continue
        
        
        if info:
            view = info[1].get_text().strip().split(' ')[-1]        # 인덱싱을 사용해 코디맵 조회수 가져오기
            blank.append(view)
        else:
            blank.append('Something is wrong')
            print('{}번째 조회수 가져올 때 문제 발생'.format(i+1))
            continue
        
        
        if info:
            reply = info[2].get_text().strip().split(' ')[-1]       # 인덱싱을 사용해 코디맵 댓글수 가져오기  
            blank.append(reply)
        else:
            blank.append('Something is wrong')
            print('{}번째 댓글수 가져올 때 문제 발생'.format(i+1))
            continue


        path = '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{}]/div[2]/a/strong'.format(str(i+1))    # (i+1)번째 코디맵 페이지로 넘어가는 XPath
        detail_page_button = driver.find_element(By.XPATH, path)    # 특정 코디맵 클릭
        detail_page_button.click()      # image 클릭은 send_keys 말고 click을 사용해야 오류가 안남 (..?)
        time.sleep(1)    

        # 페이지 로딩 관련 settings 
        interval = 1 

        prev_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            time.sleep(interval)

            curr_height = driver.execute_script("return document.body.scrollHeight")
            if curr_height == prev_height:
                break
            
            prev_height = curr_height


        # (i+1)번째 코디맵 페이지 크롤링 
        contents = BeautifulSoup(driver.page_source, 'lxml')
        content = contents.find('div', attrs={'class':'styling_content codimap-contents', 'id':'style_info'})

        
        explain = content.find('p', attrs={'class':'styling_txt'})      # (i+1)번째 코디맵 설명 가져오기 
        if explain:
            explain = explain.get_text().strip()
            blank.append(explain)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디 설명 가져올 때 문제 발생'.format(i+1))
            continue


        hashtags = content.find_all('a', attrs={'class':'ui-tag-list__item'})       # (i+1)번째 코디맵 해시태그 html 가져오기
        tag_lst = []
        for hashtage in hashtags:                   
            if hashtage:
                hashtage = hashtage.get_text().strip()
                tag_lst.append(hashtage)        # list에 hastage text들을 append하는 방식으로 저장 
            else:
                blank.append('Something is wrong')
                print('{}번째 해시태그 가져올 때 문제 발생'.format(i+1))
                continue
        blank.append(','.join(tag_lst))         # 문자열로 list 내 해시태그들 join


        image_url = content.find('img', attrs={'class':'photo'})  
        if image_url:                                                        
            image_url = image_url['src']           # (i+1)번째 코디맵 이미지 url 가져오기                            
            if image_url.startswith('//'):                                      
                image_url = 'https:' + image_url
            blank.append(image_url)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디이미지 가져올때 문제발생'.format(i+1))
            continue           
        
        driver.back()       # 뒤로 가기
        
        values.append(blank)        
        print('---------------------------------------------------')
    
    path = '/html/body/div[3]/div[3]/form/div[4]/div/div[4]/div/div/a[{}]'.format(str(page+4))      # 다음 페이지로 넘어가는 Xpath 
    next_page_button = driver.find_element(By.XPATH, path)      # 다음 페이지 클릭
    next_page_button.send_keys(Keys.ENTER)            # click 함수가 오류가 나서 send_keys로 변경
    time.sleep(3)    
        
    df = pd.DataFrame(values, columns = columns)
    codimap_list.append(df)

1페이지 크롤링 시작!/n
1페이지 1번째 코디맵 크롤링
---------------------------------------------------
1페이지 2번째 코디맵 크롤링
---------------------------------------------------
1페이지 3번째 코디맵 크롤링
---------------------------------------------------
1페이지 4번째 코디맵 크롤링
---------------------------------------------------
1페이지 5번째 코디맵 크롤링
---------------------------------------------------
1페이지 6번째 코디맵 크롤링
---------------------------------------------------
1페이지 7번째 코디맵 크롤링
---------------------------------------------------
1페이지 8번째 코디맵 크롤링
---------------------------------------------------
1페이지 9번째 코디맵 크롤링
---------------------------------------------------
1페이지 10번째 코디맵 크롤링
---------------------------------------------------
1페이지 11번째 코디맵 크롤링
---------------------------------------------------
1페이지 12번째 코디맵 크롤링
---------------------------------------------------
1페이지 13번째 코디맵 크롤링
---------------------------------------------------
1페이지 14번째 코디맵 크롤링
---------------------------------------------------
1페이지 15번째 코디맵 

In [196]:
# 페이지별로 dataframe concat & 확인

df_final = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 및 페이지별 df concat
df_final.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"16,092",49,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"#톤온톤,#뉴트럴톤,#캠퍼스,#피크닉,#와이드핏,#오버핏,#꾸안꾸,#여름,#캐주얼,...",https://image.msscdn.net/images/codimap/detail...
1,캐주얼,블랙 매니아,21.02.23,"34,681",46,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"#봄,#캐주얼,#라퍼지스토어,#다이아몬드 레이라,#나이키,#엠프렌즈,#모티패스트,#...",https://image.msscdn.net/images/codimap/detail...
2,캐주얼,오늘의 코디,22.03.22,"12,205",38,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"#로고플레이,#오버핏,#스티치,#캠퍼스,#봄,#캐주얼,#유니폼브릿지,#아디다스,#4...",https://image.msscdn.net/images/codimap/detail...
3,댄디,추천 댄디 룩,21.09.15,"20,191",27,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"#데이트,#그래픽,#레이어드룩,#가을,#댄디,#엘무드,#뮤지엄바이비컨,#테이크이지,...",https://image.msscdn.net/images/codimap/detail...
4,캐주얼,릴랙스 룩,21.02.23,"23,544",22,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"#봄,#캐주얼,#카시오,#코닥,#제로,#와릿이즌,#뉴발란스,#커버낫,#디지털,#미니...",https://image.msscdn.net/images/codimap/detail...
5,스트릿,힙하다 힙해,21.02.03,"35,295",21,레더 블루종 점퍼에 오버사이즈 니트와 조거 팬츠를 매치하고 블루 컬러 볼캡으로 멋을...,"#봄,#스트릿,#제로,#월간,#라퍼지스토어,#키치오브제,#블론드나인,#47브랜드,#...",https://image.msscdn.net/images/codimap/detail...
6,캐주얼,심플한 코디,21.07.30,"39,591",18,로고 프린팅이 돋보이는 반팔 티셔츠와 스웨트 팬츠를 매치하고 스니커즈로 완성한 이지...,"#로고플레이,#편안한,#원마일웨어,#와이드핏,#여름,#캐주얼,#리,#세미콜론 아이웨...",https://image.msscdn.net/images/codimap/detail...
7,댄디,데이트 코디,21.12.24,"10,316",18,깔끔한 무드의 니트에 데님 팬츠를 스타일링하고 스니커즈로 마무리한 댄디 룩,"#남친룩,#레더,#데이트,#테이퍼드핏,#오버핏,#소개팅,#스티치,#겨울,#댄디,#쿠...",https://image.msscdn.net/images/codimap/detail...
8,댄디,훈훈 남친 룩,21.03.24,"19,350",16,체크 패턴이 돋보이는 블레이저에 셔츠와 데님 팬츠를 더하고 담백한 구두로 마무리한 ...,"#봄,#댄디,#모디파이드,#반,#락포트,#수아레,#슈트/블레이저 재킷,#기타 시계,...",https://image.msscdn.net/images/codimap/detail...
9,캐주얼,지금 입기 좋아,21.08.31,"20,219",16,오버 핏 스웨트셔츠와 베이지 컬러 팬츠를 코디하고 더비 슈즈로 완성한 캐주얼 룩,"#꾸안꾸,#남친룩,#핀턱,#캠퍼스,#와이드핏,#여름,#캐주얼,#브래디백,#와일드 브...",https://image.msscdn.net/images/codimap/detail...


In [197]:
# dataframe 크기 확인 (8개 columns, 300개(60*5)의 코디맵)

df_final.shape

(300, 8)

In [ ]:
# save final dataframe as csv

df_final.to_csv('정지원_크롤링_challenge과제.csv', encoding = 'utf-8-sig')